In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('titanic.csv')

In [2]:
data.head(7)

,PassengerId,Survived,Pclass,Name,Gender,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S


In [3]:
def checkAdult(age):
    if age>=18:
        return "Adult"
    else:
        return "Child"
    
data["Adult/Child"]=data["Age"].apply(checkAdult)
data.head() 

,PassengerId,Survived,Pclass,Name,Gender,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Adult/Child
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Adult
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Adult
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Adult
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Adult
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Adult


In [4]:
trainingData=data[["Pclass","Adult/Child","Gender","Survived"]]
trainingData.head()

,Pclass,Adult/Child,Gender,Survived
0,3,Adult,male,0
1,1,Adult,female,1
2,3,Adult,female,1
3,1,Adult,female,1
4,3,Adult,male,0


In [5]:
def catToNum(series):
    series = series.astype('category')
#    print(series)
    return series.cat.codes

catData=trainingData[["Pclass","Adult/Child","Gender","Survived"]].apply(catToNum)
#print(catData)
trainingData[["Pclass","Adult/Child","Gender","Survived"]]=catData
trainingData.head()

C:\Users\DELL\anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Pclass,Adult/Child,Gender,Survived
0,2,0,1,0
1,0,0,0,1
2,2,0,0,1
3,0,0,0,1
4,2,0,1,0


In [6]:
trainingData = trainingData.dropna()
len(trainingData)

891

In [7]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(trainingData, test_size = 0.2)

In [8]:
train.head()

,Pclass,Adult/Child,Gender,Survived
541,2,1,0,0
505,0,0,1,0
750,1,1,0,1
358,2,1,0,1
649,2,0,0,1


In [9]:
test.head()

,Pclass,Adult/Child,Gender,Survived
345,1,0,0,1
384,2,1,1,0
396,2,0,0,0
671,0,0,1,0
433,2,1,1,0


In [10]:
len(train)

712

In [11]:
len(test)

179

In [12]:
def entropy(col):
    elements,freq = np.unique(col,return_counts = True)
    entropy = np.sum([(-freq[i]/np.sum(freq))*np.log2(freq[i]/np.sum(freq)) for i in range(len(elements))])
    return entropy


In [13]:
def InfoGain(data,split_attribute_name,target_name="Survived"):
    """
    1. data = The dataset for whose feature the IG should be calculated
    2. split_attribute_name = the name of the feature for which the information gain should be calculated
    3. target_name = the name of the target feature. The default for this example is "class"
    """    
    #Calculate the entropy of the total dataset
    parent_entropy = entropy(data[target_name])
    
    ##Calculate the entropy of the dataset
    
    #Calculate the values and the corresponding counts for the split attribute 
    vals,counts= np.unique(data[split_attribute_name],return_counts=True)
    
    #Calculate the weighted entropy
    Weighted_Entropy = np.sum([(counts[i]/np.sum(counts))*entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name]) for i in range(len(vals))])
    
    #Calculate the information gain
    Information_Gain = parent_entropy - Weighted_Entropy
    return Information_Gain
       

In [14]:
def ID3(data,originaldata,features,target_attribute_name="Survived",parent_node_class = None):
    
    
    if len(np.unique(data[target_attribute_name])) <= 1:
        return np.unique(data[target_attribute_name])[0] 
    elif len(features) ==0:
        return parent_node_class
    elif len(data)==0:
        return np.unique(originaldata[target_attribute_name])[np.argmax(np.unique(originaldata[target_attribute_name],return_counts=True)[1])]
    else:
        parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name],return_counts=True)[1])]
        
       
        values = [InfoGain(data,feature,target_attribute_name) for feature in features]
        index_of_best_feature = np.argmax(values)
        best_feature_value = features[index_of_best_feature]
        tree = {best_feature_value:{}}
        features = [i for i in features if i != best_feature_value]
        
        for value in np.unique(data[best_feature_value]):
            value = value
            sub_dataset = data.where(data[best_feature_value] == value).dropna()
            sub_tree = ID3(sub_dataset,data,features,target_attribute_name,parent_node_class)
            tree[best_feature_value][value] = sub_tree
            
        return(tree)  

In [15]:
tree = ID3(train,train,train.columns[:-1])
print(tree)

{'Gender': {0: {'Pclass': {0.0: {'Adult/Child': {0.0: 1.0, 1.0: 1.0}}, 1.0: {'Adult/Child': {0.0: 1.0, 1.0: 1.0}}, 2.0: {'Adult/Child': {0.0: 0.0, 1.0: 0.0}}}}, 1: {'Pclass': {0.0: {'Adult/Child': {0.0: 0.0, 1.0: 0.0}}, 1.0: {'Adult/Child': {0.0: 0.0, 1.0: 0.0}}, 2.0: {'Adult/Child': {0.0: 0.0, 1.0: 0.0}}}}}}
